In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import tensorflow


In [ ]:
%load_ext tensorboard

!rm -rf ./logs/

tf.config.experimental.list_physical_devices()

In [15]:
(X_train , y_train), (X_test, y_test) = keras.datasets.cifar10.load_data()

In [4]:
X_train.shape

(50000, 32, 32, 3)

In [5]:
y_train

array([[6],
       [9],
       [9],
       ...,
       [9],
       [1],
       [1]], dtype=uint8)

In [6]:
classes = ['airplane','automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [7]:
X_train.shape

(50000, 32, 32, 3)

In [8]:
X_test.shape

(10000, 32, 32, 3)

In [ ]:
X_train = np.reshape(X_train , (50000 , 32*32*3))
X_test = np.reshape(X_test , (10000 , 32*32*3))

In [ ]:
from sklearn.preprocessing import StandardScaler

X_train_scaled = StandardScaler().fit_transform(X_train)
X_test_scaled = StandardScaler().fit_transform(X_test)

In [9]:
X_train = X_train / 255
X_test = X_test / 255

In [10]:
y_train_categorical = keras.utils.to_categorical(y_train , 10, 'float32')
y_test_categorical = keras.utils.to_categorical(y_test , 10, 'float32')

In [11]:
y_train_categorical.shape

(50000, 10)

In [ ]:
%%timeit -n1 -r1

model_SGD_simple = keras.Sequential([
    keras.layers.Dense(10000, input_shape=(32*32*3,), activation='relu'),
    keras.layers.Dense(1000, activation='relu'),
    keras.layers.Dense(10, activation='sigmoid')

])

callbacker = tf.keras.callbacks.TensorBoard(log_dir='logs/model_SGD_simple', histogram_freq=1)

model_SGD_simple.compile(
    optimizer = 'SGD',
    metrics = ['accuracy'],
    loss = 'sparse_categorical_crossentropy'
)

model_SGD_simple.fit(X_train_scaled , y_train, epochs = 1, callbacks=[callbacker])

In [ ]:
%%timeit -n1 -r1

model_adam_simple = keras.Sequential([
    keras.layers.Dense(10000, input_shape=(32*32*3,), activation='relu'),
    keras.layers.Dense(1000, activation='relu'),
    keras.layers.Dense(10, activation='sigmoid')

])

callbacker = tf.keras.callbacks.TensorBoard(log_dir='logs/model_adam_simple', histogram_freq=1)

model_adam_simple.compile(
    optimizer = 'adam',
    metrics = ['accuracy'],
    loss = 'sparse_categorical_crossentropy'
)

model_adam_simple.fit(X_train_scaled , y_train, epochs = 20, callbacks=[callbacker])

In [ ]:
%%timeit -n1 -r1

model_SGD_categorical = keras.Sequential([
    keras.layers.Dense(10000, input_shape=(32*32*3,), activation='relu'),
    keras.layers.Dense(1000, activation='relu'),
    keras.layers.Dense(10, activation='sigmoid')

])

callbacker = tf.keras.callbacks.TensorBoard(log_dir='logs/model_SGD_categorical', histogram_freq=1)

model_SGD_categorical.compile(
    optimizer = 'SGD',
    metrics = ['accuracy'],
    loss = 'categorical_crossentropy'
)

model_SGD_categorical.fit(X_train_scaled , y_train_categorical, epochs = 20, callbacks=[callbacker])

In [ ]:
%%timeit -n1 -r1

model_adam_categorical = keras.Sequential([
    keras.layers.Dense(10000, input_shape=(32*32*3,), activation='relu'),
    keras.layers.Dense(1000, activation='relu'),
    keras.layers.Dense(10, activation='sigmoid')

])

callbacker = tf.keras.callbacks.TensorBoard(log_dir='logs/model_adam_categorical', histogram_freq=1)

model_adam_categorical.compile(
    optimizer = 'adam',
    metrics = ['accuracy'],
    loss = 'categorical_crossentropy'
)

model_adam_categorical.fit(X_train_scaled , y_train, epochs = 20, callbacks=[callbacker])

%%timeit -n1 -r1
with tf.device('/CPU:0'):
    cpu_model = get_model()
    cpu_model.fit(X_train_scaled, y_train_categorical, epochs=1)

In [12]:
%%timeit -n1 -r1

cnn = keras.Sequential([
    keras.layers.Conv2D(filters = 32, kernel_size = (3,3), activation = 'relu', input_shape=(32,32,3,)),
    keras.layers.MaxPooling2D((2,2)),
    keras.layers.Flatten(),

    keras.layers.Dense(64, activation = 'relu'),
    keras.layers.Dense(10, activation = 'sigmoid')
])

In [13]:
cnn.compile(
    optimizer = 'adam',
    metrics = ['accuracy'],
    loss = 'categorical_crossentropy'
)

In [ ]:
callbacker = tf.keras.callbacks.TensorBoard(log_dir='logs/with_CNN_model_adam_categorical', histogram_freq=1)

In [14]:
cnn.fit(X_train , y_train_categorical, epochs = 30 ,callbacks=[callbacker])

Epoch 1/30
1563/1563 [==============================] - 27s 12ms/step - loss: 1.4420 - accuracy: 0.4854
Epoch 2/30
1563/1563 [==============================] - 26s 17ms/step - loss: 1.1633 - accuracy: 0.5916
Epoch 3/30
1563/1563 [==============================] - 23s 15ms/step - loss: 1.0459 - accuracy: 0.6318
Epoch 4/30
1563/1563 [==============================] - 22s 14ms/step - loss: 0.9556 - accuracy: 0.6636
Epoch 5/30
1563/1563 [==============================] - 20s 13ms/step - loss: 0.8872 - accuracy: 0.6892
Epoch 6/30
1563/1563 [==============================] - 17s 11ms/step - loss: 0.8202 - accuracy: 0.7112
Epoch 7/30
1563/1563 [==============================] - 17s 11ms/step - loss: 0.7699 - accuracy: 0.7298
Epoch 8/30
1563/1563 [==============================] - 17s 11ms/step - loss: 0.7161 - accuracy: 0.7477
Epoch 9/30
1563/1563 [==============================] - 18s 11ms/step - loss: 0.6666 - accuracy: 0.7646
Epoch 10/30
1563/1563 [==============================] - 17s 11m

In [ ]:
%tensorboard --logdir logs/